In [1]:
import pandas as pd
import numpy as np

In [2]:
def ReadCSVData(path):    
    return pd.read_csv(path)

In [3]:
Data = ReadCSVData('E:\Dhaval\Elastic Search\Python\E-Book\Data File\highstoragesystem-data-for-energy-optimization\HRSS_normal_optimized.csv')

In [4]:
def add_sampleColumn(hsNData):
    result = []
    cnt = 0
    for index, value in enumerate(hsNData['Timestamp']):
        if index == 0:
            result.append(cnt)
        elif index == len(hsNData['Timestamp'])-1:
            result.append(cnt)
        elif hsNData['Timestamp'][index] - hsNData['Timestamp'][index - 1] < 0:        
            result.append(cnt+1)
            cnt += 1
        else:
            result.append(cnt)
    hsNData['sampleNum'] = result
    return hsNData, set(result)

In [5]:
Data = add_sampleColumn(Data)

In [6]:
def resampleDataperSample(Sampledata, noOfSamples, noOfTimeStamps, attributes):  
    data_sampled=np.zeros([noOfSamples, noOfTimeStamps, attributes]) # strictly speaking w/o the first 3 columns! only the 18 attributes
    time_steps= np.arange(noOfTimeStamps)*0.05 
    time_steps
    data_org= Sampledata
    time_index = np.zeros([noOfTimeStamps,])
    time_lower = np.zeros([noOfTimeStamps,])
    time_upper = np.zeros([noOfTimeStamps,])
    alpha = np.zeros([noOfTimeStamps,])
    for i in range(noOfTimeStamps):
        time_index[i] = np.sum(((np.array(data_org['Timestamp'])- time_steps[i])<=0)) -1 
    time_index.astype(int)
    time_lower = np.array(data_org['Timestamp'].loc[time_index]) #there should be no nans
    time_upper = np.array(data_org['Timestamp'].loc[time_index+1]) # last may contain nans, to be catched later
    alpha = (time_upper - time_steps)/(time_upper-time_lower)
    data_values=np.array(data_org)

    # rewrite in matrix form, will be faster
    for i in range(noOfTimeStamps):
        for j in range(attributes):
            tt=(time_index[i]+1).astype(int)
            if (time_index[i]+1)>len(data_org)-1:
                tt=len(data_org)-1
            data_sampled[1,i,j]=alpha[i]*data_values[time_index[i].astype(int),j] + (1-alpha[i])*data_values[tt,j]
    return pd.DataFrame(data_sampled[1,:,:])
    # finally catch NaNs, set to zero
    # interate through samples
    # noOfSamples no. of samples, noOfTimeStamps no. of time stamps and attributes no. of attributes
    # len(data_org)-1 no of time stamps in original data/ no of rows per sample

In [7]:
Data2 = resampleDataperSample(Data[0], max(Data[1]), 241, len(Data[0].columns))

In [8]:
def Resample_AllSamples(data,samples,_noOfTimeStamps):
    MainDataFrame = pd.DataFrame(columns=np.arange(len(data.columns)))
    for i in range(samples):
        data2 = data.loc[data['sampleNum'] == i]
        data2.index = np.arange(len(data2))
        tempdataframe = resampleDataperSample(data2, samples-1, _noOfTimeStamps, len(data2.columns))
        MainDataFrame = MainDataFrame.append(tempdataframe, ignore_index=False, verify_integrity=False, sort=None)
    MainDataFrame.index=np.arange(len(MainDataFrame))
    MainDataFrame = MainDataFrame.dropna(axis=0, how='all')
    return MainDataFrame

In [9]:
FinalData = Resample_AllSamples(Data[0],max(Data[1])+1,241)

C:\Users\Snehal.Waman\python\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  
